## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-05-14-50-02 +0000,bbc,Home secretary wants police to get more powers...,https://www.bbc.com/news/articles/c24rmdngrrjo...
1,2025-10-05-14-49-26 +0000,nyt,"Strong-Armed by Trump, Netanyahu Embraces Gaza...",https://www.nytimes.com/2025/10/05/world/middl...
2,2025-10-05-14-48-54 +0000,nypost,Unhinged man gunned down after charging office...,https://nypost.com/2025/10/05/us-news/californ...
3,2025-10-05-14-45-55 +0000,nypost,Dems shamelessly back Virginia AG nominee Jay ...,https://nypost.com/2025/10/05/us-news/dems-bac...
4,2025-10-05-14-41-00 +0000,wsj,President Trump pressed ahead with plans to mo...,https://www.wsj.com/politics/policy/trump-auth...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2315/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
8,trump,29
11,gaza,17
12,deal,12
156,will,9
133,plan,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
4,2025-10-05-14-41-00 +0000,wsj,President Trump pressed ahead with plans to mo...,https://www.wsj.com/politics/policy/trump-auth...,109
134,2025-10-04-20-36-56 +0000,nypost,Israel agrees to ‘initial withdrawal line’ in ...,https://nypost.com/2025/10/04/world-news/israe...,93
14,2025-10-05-14-02-01 +0000,nyt,Israel and Hamas Prepare for Talks on Trump’s ...,https://www.nytimes.com/2025/10/05/world/middl...,86
159,2025-10-04-17-06-03 +0000,nypost,Trump warns Hamas ‘all bets will be off’ if Ga...,https://nypost.com/2025/10/04/us-news/trump-wa...,79
155,2025-10-04-17-42-17 +0000,nypost,Trump reveals Netanyahu phone call that moved ...,https://nypost.com/2025/10/04/world-news/trump...,77


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
4,109,2025-10-05-14-41-00 +0000,wsj,President Trump pressed ahead with plans to mo...,https://www.wsj.com/politics/policy/trump-auth...
134,64,2025-10-04-20-36-56 +0000,nypost,Israel agrees to ‘initial withdrawal line’ in ...,https://nypost.com/2025/10/04/world-news/israe...
165,40,2025-10-04-16-28-25 +0000,nypost,London police arrest dozens of protesters agai...,https://nypost.com/2025/10/04/world-news/londo...
112,34,2025-10-04-23-05-09 +0000,nypost,Youngkin calls for embattled Dem AG hopeful Ja...,https://nypost.com/2025/10/04/us-news/virginia...
111,32,2025-10-04-23-14-00 +0000,wsj,Defense Secretary Pete Hegseth’s emphasis on p...,https://www.wsj.com/politics/national-security...
168,31,2025-10-04-16-00-00 +0000,wsj,"To hear the top House Democrat tell it, beyond...",https://www.wsj.com/politics/policy/hakeem-jef...
8,25,2025-10-05-14-22-56 +0000,cbc,Large-scale nighttime Russian strike on Ukrain...,https://www.cbc.ca/news/world/ukraine-russia-s...
125,23,2025-10-04-21-15-20 +0000,cbc,Canada needs co-ordinated action plan to comba...,https://www.cbc.ca/news/politics/canada-action...
26,22,2025-10-05-12-22-12 +0000,nypost,"2 killed, 14 injured in Montgomery, Ala. mass ...",https://nypost.com/2025/10/05/us-news/2-killed...
101,20,2025-10-05-00-58-12 +0000,bbc,"'They're not just sharing needles, they're sha...",https://www.bbc.com/news/articles/c0m42dwvlk8o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
